In [1]:
import json
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate

In [2]:
import os
from dotenv import load_dotenv
from langsmith import Client
from langchain_core.tracers import LangChainTracer

# .env 파일 로드
load_dotenv()

# ✅ 환경 변수 불러오기 상태 확인
print("✅ OpenAI 키 로드됨:", os.getenv("OPENAI_API_KEY") is not None)
print("✅ LangSmith 키 로드됨:", os.getenv("LANGSMITH_API_KEY") is not None)

# LangSmith 환경 설정 (동적 설정)
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGSMITH_API_KEY")  # .env에서 불러옴
os.environ["LANGCHAIN_ENDPOINT"] = os.getenv("LANGSMITH_ENDPOINT") or "https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"] = "Test"  # 원하는 프로젝트 이름

# LangSmith 클라이언트 직접 사용할 수도 있음
client = Client()
print("현재 LangSmith 프로젝트:", os.environ["LANGCHAIN_PROJECT"])


✅ OpenAI 키 로드됨: True
✅ LangSmith 키 로드됨: True
현재 LangSmith 프로젝트: Test


In [3]:
import pandas as pd
absolute_path = r"C:\Users\user\vscode\langchain\data\serviceDetail_all.csv"
df = pd.read_csv(absolute_path)  # 파일 경로에 맞게 조정
print(df.columns)  # 어떤 컬럼이 있는지 확인

Index(['구비서류', '문의처', '법령', '서비스ID', '서비스명', '서비스목적', '선정기준', '소관기관명', '수정일시',
       '신청기한', '신청방법', '온라인신청사이트URL', '자치법규', '접수기관명', '지원내용', '지원대상', '지원유형',
       '행정규칙'],
      dtype='object')


In [4]:
from langchain_core.documents import Document

def clean_text(text):
    return str(text).replace('\r', ' ').replace('\n', ' ').replace('○', '').strip()

def extract_chunks_from_row(row):
    service_name = clean_text(row.get("서비스명", ""))
    service_id = row.get("서비스ID", "")
    base_metadata = {"서비스ID": service_id}

    #  확장된 필드 리스트
    fields = ['지원대상', '지원내용', '신청방법', '접수기관명', '선정기준','문의처']

    chunks = []

    for field in fields:
        value = clean_text(row.get(field, ""))
        if value and value.lower() != "nan":
            chunk_text = f"[정책명: {service_name}] [항목: {field}]\n{value}"
            chunks.append(Document(page_content=chunk_text, metadata=base_metadata))

    return chunks

In [5]:
# 전체 데이터에서 Document 리스트 생성
documents = []
for _, row in df.iterrows():
    documents.extend(extract_chunks_from_row(row))

print(f"총 {len(documents)}개의 문서 생성 완료")

총 42795개의 문서 생성 완료


In [6]:
# OpenAIEmbeddings는 OPENAI_API_KEY를 자동으로 .env에서 불러옴
embeddings = OpenAIEmbeddings()

In [7]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
split_documents = text_splitter.split_documents(documents)

import tiktoken

def estimate_embedding_cost(docs, model="text-embedding-3-small", price_per_1k=0.00002):
    """
    문서 리스트에 대한 총 토큰 수 및 예상 비용 계산

    Args:
        docs: LangChain Document 리스트
        model: 사용할 임베딩 모델명 (기본: text-embedding-3-small)
        price_per_1k: 1K 토큰당 비용 (달러)

    Returns:
        total_tokens, estimated_cost
    """
    encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")  # 대부분 동일 토크나이저 사용
    total_tokens = sum(len(encoding.encode(doc.page_content)) for doc in docs)
    estimated_cost = (total_tokens / 1000) * price_per_1k
    return total_tokens, estimated_cost

# 사용 예시
tokens, cost = estimate_embedding_cost(split_documents)
print(f"🧮 총 토큰 수: {tokens:,}")
print(f"💸 예상 임베딩 비용: ${cost:.6f} USD")

🧮 총 토큰 수: 5,044,564
💸 예상 임베딩 비용: $0.100891 USD


In [8]:
# 문서 임베딩 후 FAISS 저장소에 저장
vectorstore = FAISS.from_documents(documents=documents, embedding=embeddings)

print("FAISS 벡터스토어 생성 완료!")

FAISS 벡터스토어 생성 완료!


In [9]:
vectorstore.save_local("faiss_index_v2")
print("✅ FAISS 저장 완료 (v2)")

✅ FAISS 저장 완료 (v2)


In [13]:
vectorstore = FAISS.load_local(
    "faiss_index",
    embeddings,
    allow_dangerous_deserialization=True
)
retriever = vectorstore.as_retriever()

In [14]:
import numpy as np
from typing import List

def maximal_marginal_relevance(
    query_embedding: np.ndarray,
    doc_embeddings: np.ndarray,
    lambda_mult: float = 0.5,
    k: int = 4
) -> List[int]:
    """Maximal Marginal Relevance (MMR) 알고리즘 직접 구현"""
    if isinstance(query_embedding, list):
        query_embedding = np.array(query_embedding)

    if isinstance(doc_embeddings, list):
        doc_embeddings = np.array(doc_embeddings)

    doc_embeddings = doc_embeddings / np.linalg.norm(doc_embeddings, axis=1, keepdims=True)
    query_embedding = query_embedding / np.linalg.norm(query_embedding)

    similarity_to_query = np.dot(doc_embeddings, query_embedding)
    similarity_between_docs = np.dot(doc_embeddings, doc_embeddings.T)

    selected = []
    remaining = list(range(len(doc_embeddings)))

    for _ in range(k):
        if not remaining:
            break

        if not selected:
            selected_idx = int(np.argmax(similarity_to_query))
            selected.append(selected_idx)
            remaining.remove(selected_idx)
            continue

        max_score = -np.inf
        selected_idx = -1

        for idx in remaining:
            sim_to_query = similarity_to_query[idx]
            sim_to_selected = max(similarity_between_docs[idx][j] for j in selected)
            score = lambda_mult * sim_to_query - (1 - lambda_mult) * sim_to_selected

            if score > max_score:
                max_score = score
                selected_idx = idx

        selected.append(selected_idx)
        remaining.remove(selected_idx)

    return selected

In [ ]:
from langchain_community.vectorstores.faiss import FAISS
from langchain_core.documents import Document
from typing import List
import numpy as np

def hybrid_mmr_retriever(
    question: str,
    vectorstore: FAISS,
    embeddings,
    top_k_sim: int = 15,  # Step 1: similarity로 후보 추출
    top_k_final: int = 5,  # Step 2: 그중 MMR로 n개 선택
    lambda_mult: float = 0.5
) -> List[Document]:
    """
    Hybrid MMR retriever:
    1. top_k_sim 문서를 similarity로 먼저 추출하고
    2. 그 중 top_k_final 문서를 MMR 방식으로 재선택
    """
    query_embedding = embeddings.embed_query(question)

    # Step 1: similarity 기반 top-k 후보 문서 가져오기
    sim_docs_and_scores = vectorstore.similarity_search_with_score_by_vector(
        query_embedding,
        k=top_k_sim
    )

    docs = [doc for doc, _ in sim_docs_and_scores]
    doc_embeddings = [embeddings.embed_query(doc.page_content) for doc in docs]

    # Step 2: MMR 알고리즘 적용
    selected_indices = maximal_marginal_relevance(
        np.array(query_embedding),
        np.array(doc_embeddings),
        lambda_mult=lambda_mult,
        k=top_k_final,
    )

    return [docs[i] for i in selected_indices]


In [16]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template("""
You are an assistant for answering questions about Korean government support policies.
Use the following retrieved context to answer the user's question.
If none of the relevant information is found in the context, say "잘 모르겠습니다." Otherwise, do not include this phrase.

When answering, try to include:
- 지원대상 (who can apply)
- 지역 또는 관할기관 (where this applies / which region or city is responsible)

Use the following output format for each policy:

사업명: ...
요약: ...
신청 URL 또는 문의처: ...

- If the policy has a 신청 URL, include it as "신청 URL: ..."
- If there is no URL, include the 문의처 (e.g., 전화번호 or 접수기관명) instead.

Respond in Korean.

# Context:
{context}

# Question:
{question}

# Answer:
""")

In [17]:
def format_markdown_response(text: str) -> str:
    # 간단한 Markdown 후처리 예시
    lines = text.strip().split("\n")
    md_lines = []

    for line in lines:
        if line.startswith("사업명:"):
            md_lines.append(f"**{line}**")
        elif line.startswith("요약:"):
            md_lines.append(f"📍 {line}")
        elif line.startswith("신청 URL"):
            url = line.split(":", 1)[-1].strip()
            md_lines.append(f"🔗 [신청 바로가기]({url})")
        elif line.startswith("문의처:"):
            md_lines.append(f"📞 {line}")
        else:
            md_lines.append(line)

    return "\n\n".join(md_lines)

In [18]:
# LangSmith 트레이싱은 .env 설정만으로 자동 활성화됨
# LANGSMITH_TRACING=true 설정 시 실행 로그를 LangSmith에서 확인 가능

llm = ChatOpenAI(model_name="gpt-4o", temperature=0)

In [19]:
def hybrid_chain(question: str) -> str:
    docs = hybrid_mmr_retriever(
        question=question,
        vectorstore=vectorstore,
        embeddings=embeddings,
        top_k_sim=15,
        top_k_final=5,
        lambda_mult=0.7
    )
    context = "\n\n".join([doc.page_content for doc in docs])
    formatted_prompt = prompt.format(context=context, question=question)
    response = llm.invoke(formatted_prompt)
    answer = response.content if hasattr(response, "content") else response

    markdown_result = format_markdown_response(answer)
    return markdown_result


In [20]:
# similarity 기반 retriever
retriever_sim = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 5}
)

# mmr 기반 retriever
retriever_mmr = vectorstore.as_retriever(
    search_type="mmr",
    search_kwargs={"k": 5, "lambda_mult": 0.7}
)

In [21]:
from langchain_core.runnables import RunnableLambda

# 후처리 Runnable 래핑
markdown_formatter = RunnableLambda(format_markdown_response)

# 체인 구성
chain_sim = (
    {"context": retriever_sim, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
    | markdown_formatter  
)

chain_mmr = (
    {"context": retriever_mmr, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
    | markdown_formatter  
)


## TEST

In [22]:
def compare_retrievers(question: str):
    print(f"\n❓ 질문: {question}")
    
    # --- similarity 체인 실행 ---
    response_sim = chain_sim.invoke(question)
    print("\n🔹 similarity 방식 응답:\n", response_sim)

    # --- mmr 체인 실행 ---
    response_mmr = chain_mmr.invoke(question)
    print("\n🔸 mmr 방식 응답:\n", response_mmr)

    # --- hybrid mmr 체인 실행 ---
    response_hybrid = hybrid_chain(question)
    print("\n🌀 hybrid MMR 방식 응답:\n", response_hybrid)


In [23]:
compare_retrievers("현재 구직 중인 청년이 신청할 수 있는 정부 지원 정책이 뭐가 있어?")


❓ 질문: 현재 구직 중인 청년이 신청할 수 있는 정부 지원 정책이 뭐가 있어?

🔹 similarity 방식 응답:
 **사업명: 청년 취업활동수당  **

📍 요약: 영광군에 거주하는 만 18세부터 45세까지의 미취업 청년에게 매월 50만원씩 최대 6개월 동안 취업활동 수당을 지원합니다.  

지원대상: 영광군 거주, 만 18세∼45세, 가구소득인정액이 기준중위소득 150% 이하인 미취업 청년  

지역 또는 관할기관: 전라남도 영광군  

🔗 [신청 바로가기]([정부24](https://www.gov.kr/portal/rcvfvrSvc/dtlEx/497000000108))

📞 문의처: 인구교육정책실/061-350-5197  



**사업명: 청년 취업지원 희망프로젝트  **

📍 요약: 제주특별자치도 내 중소기업이 15세~39세 이하의 미취업 청년을 채용할 경우, 인건비 일부를 지원합니다.  

지원대상: 15세~39세 이하 미취업청년을 채용한 도내 중소기업  

지역 또는 관할기관: 제주특별자치도  

🔗 [신청 바로가기]([정부24](https://www.gov.kr/portal/rcvfvrSvc/dtlEx/650000000317))

📞 문의처: 경제일자리과/064-710-3795, 064-710-3797

🔸 mmr 방식 응답:
 **사업명: 청년 취업활동수당  **

📍 요약: 영광군에 거주하는 만 18세∼45세의 미취업 청년으로, 가구소득인정액이 기준중위소득 150% 이하인 경우 매월 50만원씩 최대 6개월 동안 취업활동 수당을 지원받을 수 있습니다.  

🔗 [신청 바로가기]([정부24](https://www.gov.kr/portal/rcvfvrSvc/dtlEx/497000000108))



**사업명: 청년 공공근로 지원  **

📍 요약: 서천군에 거주하는 만 18세 이상의 주민으로, 기준 중위소득 70% 이하이면서 재산 4억원 이하인 근로능력이 있는 청년은 공공기관 행정사무 보조 인력으로 채용될 수 있습니다.  

📞 문의처: 서

In [72]:
compare_retrievers("3세 이하의 자녀가 있는데 지원가능한 정부지원사업 알려줘")


❓ 질문: 3세 이하의 자녀가 있는데 지원가능한 정부지원사업 알려줘

🔹 similarity 방식 응답:
 사업명: 둘째이후 자녀 보육료 지원  
요약: 통영시에 부모와 함께 3개월 이상 계속하여 주민등록이 되어 있고 실제 거주하는 둘째 이후 자녀를 둔 가정에 보육료를 지원합니다. 1세부터 3세까지의 자녀에 대해 기본보육료 지원단가의 80%에서 정부지원금액을 차감한 차액을 지원합니다.  
지원대상: 통영시에 부모와 함께 3개월 이상 계속하여 주민등록이 되어 있고 실제 거주하는 둘째 이후 자녀를 둔 부 또는 모  
지역 또는 관할기관: 경상남도 통영시  
신청 URL 또는 문의처: 여성가족과/055-650-4633  

사업명: 법정저소득·셋째아 필요경비 지원  
요약: 경기도 안양시 관내 어린이집에 재원 중인 만 5세 이하의 법정저소득 및 셋째아 이상 아동에게 특별활동비 및 현장학습비를 지원합니다.  
지원대상: 만0~5세 어린이집 재원 중인 법정저소득 및 셋째아 이상 아동  
지역 또는 관할기관: 경기도 안양시  
신청 URL 또는 문의처: 안양시청 여성가족과/031-8045-5588  

잘 모르겠습니다.

🔸 mmr 방식 응답:
 잘 모르겠습니다.

🌀 hybrid MMR 방식 응답:
 사업명: 만3~5세 부모부담보육료 지원
요약: 만3~5세 아동이 민간, 가정, 정부미지원 사회복지법인, 협동 및 법인, 단체어린이집, 공공형어린이집에 재원 중일 경우, 어린이집 보육료 수압액과 정부지원 보육료의 차액을 지원합니다. 매월 74,000원 ~ 90,000원이 지원됩니다.
신청 URL 또는 문의처: 영천시 가족행복과/054-330-6213

사업명: 둘째이후 자녀 보육료 지원
요약: 통영시에 부모와 함께 3개월 이상 계속하여 주민등록이 되어 있고 실제 거주하는 둘째 이후 자녀를 둔 가정에 대해 보육료를 지원합니다. 1세(2023년생) 자녀의 경우 280,000원, 2세(2022년생) 자녀의 경우 215,200원, 3세(2021년생) 자녀의 경우 124,000원을

In [90]:
compare_retrievers("아직 유치원을 가지않은 자녀가 있는데 지원가능한 정부지원사업 알려줘")


❓ 질문: 아직 유치원을 가지않은 자녀가 있는데 지원가능한 정부지원사업 알려줘

🔹 similarity 방식 응답:
 **사업명: 어린이집·유치원 신입생 입학준비금 지원  **

📍 요약: 어린이집·유치원에 최초 입학하는 영유아 가정에 입학준비금 10만원을 지원합니다.  

지원대상: 입학일 기준 부모 또는 보호자와 함께 수영구에 주소를 두고 2022. 1. 1 이후 어린이집·유치원에 최초 입학하는 영유아. 가정양육에서 최초로 어린이집 또는 유치원으로 변경된 경우에만 지원 가능합니다.  

지역 또는 관할기관: 부산광역시 수영구  

🔗 [신청 바로가기](수영구청 가족행복과/051-610-4326)

상세조회URL: [https://www.gov.kr/portal/rcvfvrSvc/dtlEx/338000000122](https://www.gov.kr/portal/rcvfvrSvc/dtlEx/338000000122)  



**사업명: 거제시 유치원생 및 초등학생 입학축하금 지원  **

📍 요약: 거제시에 주소를 두고 유치원 및 초등학교에 최초 입학하는 아동의 가정에 입학축하금을 지원합니다.  

지원대상: 입학일 기준 거제시에 주소를 두고 유치원 및 초등학교에 최초 입학하는 학생.  

지역 또는 관할기관: 경상남도 거제시  

🔗 [신청 바로가기](거제시 평생교육과/055-639-3854)

상세조회URL: [https://www.gov.kr/portal/rcvfvrSvc/dtlEx/537000001435](https://www.gov.kr/portal/rcvfvrSvc/dtlEx/537000001435)

🔸 mmr 방식 응답:
 **사업명: 외국국적 유아학비 지원**

📍 요약: 공·사립 유치원에 재원 중인 외국국적 3~5세 유아에게 학비를 지원하는 사업입니다.

지원대상: 공·사립 유치원에 재원 중인 외국국적 3~5세 유아

지역 또는 관할기관: 전라남도

🔗 [신청 바로가기](문의전화: 유초등교육과/061-260-0396)



**사업명: 거제